<a href="https://colab.research.google.com/github/Minahil-official/Quater-2/blob/main/video_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install --upgrade --quiet google.genai

In [39]:
!pip install gTTS --quiet

In [13]:
!pip install playsound

In [15]:
import google.generativeai as genai

In [16]:
from google.colab import userdata
GEMINI_API_KEY: str = userdata.get('video_process')
if(GEMINI_API_KEY):
  print("API Key found")
else:
  print("API Key not found")

API Key found


In [17]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)

model: str ="gemini-2.0-flash-exp"

In [18]:
# This appears to be a file path and not Python code.  No correction needed.
# If you intend to use this in Python code, you'll likely need to put it in quotes:
video_file_name = "/content/WhatsApp Video 2024-12-24 at 18.23.08.mp4"
#  (Note the double backslashes for escaping backslashes in Python strings)

In [19]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)
  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file

my_video = upload_video(video_file_name)

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/23ejnds2zwt7


In [23]:
from google.genai.types import Content, Part
prompt = """For each scene in this video,
            generate captions that describe the scene along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = my_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                ]),
        prompt,
    ]
)
print(response.text)

```json
[
  {
    "timecode": "0:00",
    "caption": "A person's hand is visible in the frame, with their thumb pointed upward. The background includes a patterned rug and a dark-colored laptop."
  },
  {
    "timecode": "0:01",
     "caption": "The hand moves and the fingers are extended. The background remains the patterned rug and a dark-colored laptop."
  },
   {
    "timecode": "0:02",
    "caption": "The hand is seen with the fingers slightly extended. The background includes a patterned rug and a dark-colored laptop."
  },
 {
    "timecode": "0:03",
    "caption":"The hand moves with the fingers extended and visible. The background includes a patterned rug and a dark-colored laptop."
  },
   {
    "timecode": "0:05",
    "caption": "The fingers are slightly closed as the hand moves. The background includes a patterned rug and a dark-colored laptop."
   },
   {
    "timecode": "0:06",
     "caption": "The fingers are slightly closed as the hand moves. The background includes a pa

In [38]:
from gtts import gTTS
from playsound import playsound
from IPython.display import Audio

# ... (Your code to extract text from the response) ...

tts = gTTS(text= "\"Hello, my name is Minahil Nawaz and I am a student at PIAIC Lahore and I'm learning agentic AI nowadays.\""

, lang='en')  # Create a gTTS object with the extracted text

# Save the audio to a file
with open("output.mp3", "wb") as f:
    for chunk in tts.stream():
        f.write(chunk)

# Play the audio using IPython.display Audio to ensure it works in Colab
# This is more reliable than playsound in Colab environments.
display(Audio("output.mp3", autoplay=True))